# El objetivo general de esta prueba es alcanzar el mejor desempeño posible para clasificar si un tweet es positivo o negativo.
## Para medir el desempeño, se evaluará con un conjunto de datos del cuál no tendrán acceso. De esta manera evitaremos que los modelos aprendan información sobre el conjunto de validación.
### Crea una carpeta de trabajo y guarda todos los archivos correspondientes (notebook, archivos auxiliares y csv). Una vez terminada la prueba, comprime la carpeta y sube el .zip a la sección correspondiente.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
import random
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stap = stopwords.words("english")
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/moisesquiroz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/moisesquiroz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import string

In [3]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# 1.-Generar un análisis exploratorio sobre los datos contenidos en el DataFrame, considerando palabras más comunes y distribución de las clases en el vector objetivo. 

In [4]:
#LO QUE HAREMOS, SERÁ CREAR MODELOS DE REGRESIÓN LOGÍSTICA, BAYES, ARBOLES DE DECISIÓN, RANDOMFORESTCLASS, SVM, LDA, Adaboost Y Gradiente BUSCANDO LOS MEJORES HIPERPARÁMETROS CON GRIDSEARCH. 

In [27]:
nombre_archivo = input("ingrese el nombre del archivo:")

ingrese el nombre del archivo:training_tweets.csv


In [28]:
df = pd.read_csv(nombre_archivo).drop(columns=["Unnamed: 0"])

##### 3. Preparación del vector objetivo y las matrices de entrenamiento y validación:


In [ ]:
df['sentiment'] = df['sentiment'].map({'love':1,'worry':-1,'happiness':1,'fun':1,'hate':-1,'sadness':-1,'empty':-1,'relief':1,'surprise':1,'enthusiasm':1,'boredom':-1,'anger':-1})

In [6]:
def cambiar_de_a_poco(Dataframe, col):
    from nltk.corpus import stopwords
    stap = stopwords.words("english")
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stap)])) #Saca stopwords
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '. join([word.lower() for word in x.split()])) #minúsculas
    Dataframe[col] = pd.Series(map(lambda x: re.sub("((www\.[^\s]+)|(https?://[^\s]+))", "", x), Dataframe[col])) #Convierte url's 
    Dataframe[col] = pd.Series(map(lambda x: re.sub("@[^\s]+", "", x), Dataframe[col])) #Elimina usuarios
    Dataframe[col] = pd.Series(map(lambda x: re.sub(r"#([^\s]+)", r"\1", x), Dataframe[col])) #Saca los Hashtags
    Dataframe[col] = pd.Series(map(lambda x: re.sub('[\s]+', ' ', x), Dataframe[col]))  #Elimina dobles espacios
    Dataframe[col] = pd.Series(map(lambda x: "".join([char for char in x if char not in string.punctuation]), Dataframe[col])) #Eliminar puntuación
    Dataframe[col] = pd.Series(map(lambda x: "".join([char for char in x if char not in string.punctuation]), Dataframe[col])) #Eliminar puntuación
    Dataframe[col] = Dataframe[col].apply(lambda x: ' '. join([lmtzr.lemmatize(word) for word in x.split()])) #Generaliza palabras

In [20]:
cambiar_de_a_poco(df, "content")

In [ ]:
df.head(6)

In [22]:
df2 = df.drop(index=df[df.sentiment.isna()].index)

#### se borraron los index donde son null

In [ ]:
df2.head(6)

In [ ]:
sns.distplot(df2["sentiment"], kde=False)

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stap, lowercase=True)
count_vectorizer_fit = count_vectorizer.fit_transform(df2["content"])
words = count_vectorizer.get_feature_names()
words_freq = count_vectorizer_fit.toarray().sum(axis=0)
aux = pd.DataFrame([words, words_freq]).T
aux.columns = ["Palabra", "Freq"]
aux = aux.sort_values(by=['Freq'],ascending=False)
plt.figure(figsize=(15,25))
plt.barh(y=aux["Palabra"][:100], width=aux["Freq"][:100])

In [16]:
def counterizar(DataFrame, colname):
    countvec = CountVectorizer(stop_words=stap, lowercase=True)
    countvec_fit = countvec.fit_transform(DataFrame[colname])
    return countvec_fit

In [ ]:
#Las palabras más utilizadas que puedes servir para discriminar el sentimiento que hay detrás de las palabras.

####  Entrenamiento de modelos: 
    En base a los modelos vistos en clase, implemente por lo menos 5. Para cada uno de ellos justifique la elección de hiperparámetros. Si implementa búsqueda de grilla para cada uno de ellos, defina el rango de valores a tomar en cada hiperparámetro. 
    Reporte el desempeño de cada modelo en las muestras de entrenamiento y validación. Comente sobre la capacidad de generalización de cada uno de ellos haciendo uso de los conceptos vistos en el curso. 

In [ ]:
reglog = LogisticRegression(n_jobs=-1, random_state=2503)
naiveBayes = BernoulliNB(fit_prior=False)
TreeClass = DecisionTreeClassifier(random_state=2503)
SVCClass = SVC(random_state=2503)
RandomClass = RandomForestClassifier(random_state=2503, n_jobs=-1)
modelo_adaboost = AdaBoostClassifier(random_state=253)
modelo_gradiente = GradientBoostingClassifier(random_state=253)

In [ ]:
paramsreglog = {"C":[0.1, 0.5, 0.75, 1, 1.25, 1.5, 2], "fit_intercept": [True, False], "max_iter": [50, 100, 250, 500, 1000]}
paramsnaive = {"alpha": [0.5, 0.7, 1]}
paramsTree = {"max_depth": list(np.linspace(1,32,32)), "max_features": list(np.linspace(1,9,9, dtype=int))}
paramssvc = {"C":[0.001, 0.01, 0.1, 1, 10, 200, 800], "gamma": [0.0001, 0.001, 0.01, 0.1, 5]}
paramsrandomf = {"max_depth": [50, 65, 75, 90, 110], "n_estimators": np.linspace(20, 1000, 13, dtype=int)}
params_adaboost= {'learning_rate': [0.01, 0.1,0.3, 0.5, 0.8],'n_estimators': [50, 100, 250, 500, 1000, 2000]}
params_gradient = {'learning_rate': [0.01, 0.1,0.3,0.5, 0.8],'n_estimators': [50, 100, 250, 500, 1000, 2000],'subsample': [0.1,0.5,0.7, 0.9]}

In [ ]:
"""gridreglog = GridSearchCV(reglog, param_grid=paramsreglog, cv=10, n_jobs = -1)
gridbayes = GridSearchCV(naiveBayes, param_grid=paramsnaive, cv=10, n_jobs=-1)
gridtree = GridSearchCV(TreeClass, param_grid=paramsTree, cv=10, n_jobs=-1)
gridsvc = GridSearchCV(SVCClass, param_grid=paramssvc, cv=5, n_jobs=-1)
gridrandomf = GridSearchCV(RandomClass, param_grid=paramsrandomf, cv=3, n_jobs=-1)
gridada = GridSearchCV(modelo_adaboost, params_adaboost, cv=10, n_jobs=-1)
gridgradiente = GridSearchCV(modelo_gradiente, params_gradient, cv=10, n_jobs=-1)"""

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(count_vectorizer_fit, df2["sentiment"], test_size=.33, random_state = 2503)

In [ ]:
%%time
gridreglog.fit(x_train, y_train)

In [ ]:
%%time
gridbayes.fit(x_train, y_train)

In [ ]:
%%time
gridtree.fit(x_train, y_train)

In [ ]:
%%time
gridsvc.fit(x_train, y_train)

In [ ]:
%%time
gridrandomf.fit(x_train, y_train)

In [ ]:
%%time
gridada.fit(x_train, y_train)

In [ ]:
%%time
gridgradiente.fit(x_train, y_train)

In [ ]:
gridreglog.best_params_

In [ ]:
gridbayes.best_params_

# Gabriel:
    Para evaluar el desempeño de los modelos haz lo siguiente:
    Ejecuta las importaciones del Notebook y las funciones cambiar_de_a_poco y counterizar
    Lee tu dataframe
    Aplica la función cambiar_de_a_poco
    Importa los dos modelos
    Evalúa los modelos dándoles como conjunto de X's , la función counterizar.
    

In [ ]:
gridsvc.best_params_

In [ ]:
gridtree.best_params_

In [ ]:
gridada.best_params_

In [ ]:
gridrandomf.best_params_

In [ ]:
gridgradiente.best_params_

In [ ]:
modelo_gradiente2 = GradientBoostingClassifier(random_state=2503)
params_gradient2 = {'learning_rate': [0.045, 0.075, 0.09, 0.1, 0.15, 0.18, 0.21],'n_estimators': [1500, 2000, 2500, 3000],'subsample': [0.7, 0.9, 0.95]}
gridgradiente2 = GridSearchCV(modelo_gradiente, params_gradient2, cv=10, n_jobs=-1)

In [ ]:
%%time
gridgradiente2.fit(x_train, y_train)

In [ ]:
gridgradiente2.best_params_

In [ ]:
model_ada = gridada.best_estimator_
model_bayes = gridbayes.best_estimator_
model_gradiente = gridgradiente.best_estimator_
model_gradiente2 = gridgradiente2.best_estimator_
model_randomf = gridrandomf.best_estimator_
model_reglog = gridreglog.best_estimator_
model_svc = gridsvc.best_estimator_
model_tree = gridtree.best_estimator_

In [ ]:
y_pred_ada = model_ada.predict(x_test)
y_pred_bayes = model_bayes.predict(x_test)
y_pred_grad = model_gradiente.predict(x_test)
y_pred_grad2 = model_gradiente2.predict(x_test)
y_pred_randomf = model_randomf.predict(x_test)
y_pred_reglog = model_reglog.predict(x_test)
y_pred_svc = model_svc.predict(x_test)
y_pred_tree = model_tree.predict(x_test)

In [ ]:
print("Modelo AdaBoost")
print(classification_report(y_pred_ada, y_test, digits=4))
print("="*80)
print("Modelo Bayes")
print(classification_report(y_pred_bayes, y_test, digits=4))
print("="*80)
print("Modelo Gradiente")
print(classification_report(y_pred_grad, y_test, digits=4))
print("="*80)
print("Modelo Gradiente.v2")
print(classification_report(y_pred_grad2, y_test, digits=4))
print("="*80)
print("Modelo Random Forest")
print(classification_report(y_pred_randomf, y_test, digits=4))
print("="*80)
print("Modelo Regresión Logística")
print(classification_report(y_pred_reglog, y_test, digits=4))
print("="*80)
print("Modelo SVC")
print(classification_report(y_pred_svc, y_test, digits=4))
print("="*80)
print("Modelo Árbol Class")
print(classification_report(y_pred_tree, y_test, digits=4))

#### Seleccione los 2 mejores modelos, serialicelos y envíelos a evaluación. 
    Recuerde que el modelo serializado debe ser posterior al fit , para poder ejecutar predict en los nuevos datos. 

In [ ]:
#Los mejores dos modelos son: Regresión Logística y Bayes, ya que presentan mejor promedio de f1-score

In [ ]:
Bayes_model = 'Bayes_MJpR.sav'
pickle.dump(model_bayes, open(Bayes_model, 'wb'))

In [ ]:
Reg_log_model = 'Reglog_MJpR.sav'
pickle.dump(model_reglog, open(Reg_log_model, 'wb'))

In [11]:
#ABRIR LOS MODELOS DE LA SGTE MANERA:
import pickle
modelo_cargado = pickle.load(open("Bayes_MJpR.sav", "rb"))

modelo_cargado2 = pickle.load(open("Reglog_MJpR.sav", "rb"))

/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator BernoulliNB from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [23]:
modelo_cargado.predict(counterizar(df2,"content"))

ValueError: Expected input with 23788 features, got 27698 instead

In [13]:
df.head()

,content,sentiment
0,happy mama day mother,love
1,i lost please help find good home,worry
2,yes yes i am networking whore fullestand girl ...,happiness
3,you wish would tweet followed me,neutral
4,microeconomics project ihate subject amp besid...,worry
